In [26]:
import pandas as pd
import sqlite3
import os
import numpy as np
from datetime import datetime, timedelta,date
from google.cloud import bigquery
from binance.client import Client
from dotenv import dotenv_values

# Init Parameter and Constant variable

In [27]:
ds='sqlite'
#ds='bq'

is_py=True
required_report=False

main_statbleCoin='BUSD'

if ds=='sqlite':
    table='hist_trade'
else :
    table="`pongthorn.FinDW.bn_hist_trade`"
    
print(f"{ds} - {table}")

sqlite - hist_trade


In [28]:
buildtime = datetime.now().strftime('%d%m%y_%H%M')
path_config=r'D:\AB_DB\Script_ImportData'
root_path=r'D:\BC-World\CryptoTrans\BS-Trade'
data_base_file=r'D:\AB_DB\Script_ImportData\Database\DB_BN_MyTrade.db'

In [29]:
config_values= dotenv_values(dotenv_path=f'{path_config}\\.env')
api_key = config_values['bn_key']
api_secret =config_values['bn_secrete']
client=Client(api_key,api_secret)

In [30]:
if is_py:
    q_start = input("Enter start query(2019-01-01 01:00): ") or '2022-09-01'
    q_end = input("Enter end query(2019-01-01 01:00): ") or ''
    

    if q_start!='' and q_end!='':
        try:
            start_dt=datetime.strptime(q_start, "%Y-%m-%d")
            end_dt=datetime.strptime(q_end,"%Y-%m-%d")
            if start_dt>=end_dt:
               raise Exception(f"Error : {start_dt} can't be greater or equal to {end_dt}")
        except Exception as ex:
            raise ex


    if q_start!='' and q_end!='':
     print(f"Do you want to retrive data from {start_dt} to {end_dt} ?")
    else:
     print(f"Do you want to retrive all data ?")   
    press_y=input(f"Press y=True and n=False to process report : ") 
    if press_y.lower()=='y':
     ok=True
    else:
     exit()

Enter start query(2019-01-01 01:00):  2023-06-15
Enter end query(2019-01-01 01:00):  2023-07-04


Do you want to retrive data from 2023-06-15 00:00:00 to 2023-07-04 00:00:00 ?


Press y=True and n=False to process report :  y


In [31]:
def date_str_report(date_str):
    date_x=datetime.strptime(date_str,'%Y-%m-%d')
    return date_x.strftime('%d%b%y')

In [32]:
#date_from='2022-06-20'
if q_start=='':
  date_from=''  
  date_str_from=''
else:
 date_from=q_start
 date_str_from=date_str_report(date_from)
print(date_from)

2023-06-15


In [33]:
if q_end=='':
 date_to=date.today()
 datetime_to=datetime(date_to.year,date_to.month,date_to.day)
 date_to=date_to.strftime('%Y-%m-%d')

else:
 date_to=q_end

date_str_to=date_str_report(date_to)
datetime_to=datetime.strptime(date_to,'%Y-%m-%d') # convert str to datetime so that we can add 1 day delta
datetime_to_plus1=datetime_to+timedelta(days=1)
datet_to_plus1=datetime_to_plus1.strftime('%Y-%m-%d')

print(date_to)
print(datet_to_plus1)

2023-07-04
2023-07-05


In [34]:
print(f"{date_str_from} - {date_str_to}")

15Jun23 - 04Jul23


# Load Data Transaction

In [35]:
if  date_from=='' and date_to=='' :
 sql=f"select * from {table}   order by datetimeUTC"
elif   date_from=='' and date_to!='' :
  sql=f"select * from {table} where   datetimeUTC<'{datet_to_plus1}' order by datetimeUTC desc"  
elif date_from!='' and date_to=='' :
  sql=f"select * from {table} where datetimeUTC >='{date_from}'  order by datetimeUTC desc" 
else:
 sql=f"select * from {table} where datetimeUTC >='{date_from}' and  datetimeUTC<'{datet_to_plus1}' order by datetimeUTC desc"
print(sql)



select * from hist_trade where datetimeUTC >='2023-06-15' and  datetimeUTC<'2023-07-05' order by datetimeUTC desc


In [36]:
if ds=='sqlite':
 
 print("Connect SQLite")
 conn = sqlite3.connect(os.path.abspath(data_base_file))   
 df_trasns=pd.read_sql_query(sql, conn)

else :
 print("Connect BigQuery")   
 conn = bigquery.Client()   
 query_job =conn.query(query=sql)
 df_trasns= query_job.to_dataframe()    
    
    

Connect SQLite


In [37]:
print(df_trasns[['datetimeUTC','symbol_pair','type','price','qty','total']].head())

           datetimeUTC symbol_pair type     price     qty     total
0  2023-07-03 15:35:07     ETHBUSD  BUY   1976.12  0.0100  19.76120
1  2023-07-03 11:41:36     BTCBUSD  BUY  30638.35  0.0010  30.63835
2  2023-07-03 11:41:24     ETHBUSD  BUY   1962.37  0.0150  29.43555
3  2023-07-02 22:27:23     ETHBUSD  BUY   1951.08  0.0055  10.73094
4  2023-07-02 22:27:23     ETHBUSD  BUY   1951.16  0.0095  18.53602


In [38]:
print(df_trasns[['datetimeUTC','symbol_pair','type','price','qty','total']].tail())

            datetimeUTC symbol_pair  type     price      qty      total
19  2023-06-26 16:45:31     ETHBUSD   BUY   1855.00  0.01500  27.825000
20  2023-06-26 15:10:56     BTCBUSD   BUY  30400.97  0.00053  16.112514
21  2023-06-26 15:10:56     BTCBUSD   BUY  30400.97  0.00047  14.288456
22  2023-06-26 01:13:33     ETHBUSD   BUY   1875.00  0.01500  28.125000
23  2023-06-17 15:15:17     ETHBUSD  SELL   1732.98  0.01750  30.327150


# Find Summary Buy & Sell

In [39]:
print("Agg transaction")
groupby_cols=['symbol','type']
agg_cols=['qty','total']
df_summary=df_trasns.groupby(groupby_cols,as_index=False)[agg_cols].sum()
df_summary['avg_price']=df_summary['total']/df_summary['qty']
df_summary=df_summary.round(5)
print(df_summary)

Agg transaction
  symbol  type     qty      total    avg_price
0    BTC   BUY  0.0055  168.01014  30547.29845
1    BTC  SELL  0.0018   53.87040  29928.00000
2    ETH   BUY  0.1320  250.20842   1895.51830
3    ETH  SELL  0.0525   95.11530   1811.72000


In [40]:
print("Total Buy/Sell PL")
dfAll=df_summary.groupby(['type'],as_index=False)[['total']].sum()
if len(dfAll)==2:
 dfAll.loc[len(dfAll)] = ["NET", (round(dfAll.loc[1,'total'],1)-round(dfAll.loc[0,'total'],1) )   ]   
print(dfAll)

Total Buy/Sell PL
   type      total
0   BUY  418.21856
1  SELL  148.98570
2   NET -269.20000


In [41]:
print("Symbol in transaction")
list_sysmbols=df_summary['symbol'].unique()
print(list_sysmbols)

Symbol in transaction
['BTC' 'ETH']


In [42]:
print("Simulate Sell On Buy transactiond (Found Buy and Sell on same symbol)")
dfBSSimulation=pd.DataFrame(columns=list(df_summary.columns))
type_action=['BUY','SELL']
for symbol in list_sysmbols:
    df_bs=df_summary.query("symbol==@symbol")
    df_bs.reset_index(inplace=True,drop=True)
    if len(df_bs)==2:
       df_bs.loc[1,"qty"]=df_bs.loc[0,"qty"]
       # print(df_bs)
       dfBSSimulation=pd.concat([dfBSSimulation,df_bs]) 
dfBSSimulation['total']=dfBSSimulation['qty']*dfBSSimulation['avg_price']
dfBSSimulation=dfBSSimulation[['symbol' ,'type', 'qty' ,  'avg_price','total']]   
print(dfBSSimulation)

Simulate Sell On Buy transactiond (Found Buy and Sell on same symbol)
  symbol  type     qty    avg_price       total
0    BTC   BUY  0.0055  30547.29845  168.010141
1    BTC  SELL  0.0055  29928.00000  164.604000
0    ETH   BUY  0.1320   1895.51830  250.208416
1    ETH  SELL  0.1320   1811.72000  239.147040


In [43]:
def get_current_price(symbol):
    symbol_pair=f"{symbol}{main_statbleCoin}"
    current_price =float( client.get_symbol_ticker(symbol=symbol_pair)['price'])
    current_price=round(current_price,5)
    return current_price


print("Test Simulate Sell with Same Qty of Buy At The Current Price")
buy_action='BUY'
dfBSAll=pd.DataFrame(columns=list(df_summary.columns))

for symbol in list_sysmbols:
    df_bs=df_summary.query("symbol==@symbol and type==@buy_action")
    df_bs.reset_index(inplace=True,drop=True)
    df_bs.loc[1,"symbol"]=symbol
    df_bs.loc[1,"type"]="SELL All"
    df_bs.loc[1,"qty"]=df_bs.loc[0,"qty"]
    df_bs.loc[1,"avg_price"]=get_current_price(symbol)
    df_bs.loc[1,"total"]=df_bs.loc[1,"qty"]*df_bs.loc[1,"avg_price"]*-1
    # print(df_bs)
    dfBSAll=pd.concat([dfBSAll,df_bs]) 
    

dfBSAll=dfBSAll[['symbol' ,'type', 'qty' ,  'avg_price','total'] ]  
print(dfBSAll)

Test Simulate Sell with Same Qty of Buy At The Current Price


C:\Users\pongthsa\AppData\Local\Temp\ipykernel_14408\4100216823.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bs.loc[1,"symbol"]=symbol
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_14408\4100216823.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bs.loc[1,"symbol"]=symbol


  symbol      type     qty    avg_price       total
0    BTC       BUY  0.0055  30547.29845  168.010140
1    BTC  SELL All  0.0055  30968.01000 -170.324055
0    ETH       BUY  0.1320   1895.51830  250.208420
1    ETH  SELL All  0.1320   1957.84000 -258.434880


In [44]:
report_y=input(f"Press y to build report: ") 
if  report_y.lower()=='y' and df_trasns.empty==False :
    
    result_path=f'{root_path}\\Trade_{date_str_from}-{date_str_to}_At_{buildtime}.xlsx'
    
    writer=pd.ExcelWriter(result_path,engine='xlsxwriter') 

    df_summary.to_excel(writer, sheet_name="SummaryBuySell",index=False)   
    dfAll.to_excel(writer,sheet_name="NetAll",index=False)
    dfBSSimulation.to_excel(writer, sheet_name="BuySell_Simulation",index=False) 

    
    dfBSAll.to_excel(writer, sheet_name="BuySell_CurrentPrice",index=False) 
    df_trasns.to_excel(writer, sheet_name="Transaction",index=False)

    writer.save()
    print(f"get report at "+result_path)
else:
    print("no data to create report")




Press y to build report:  y


C:\Users\pongthsa\AppData\Local\Temp\ipykernel_14408\1714457520.py:16: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()


get report at D:\BC-World\CryptoTrans\BS-Trade\Trade_15Jun23-04Jul23_At_030723_2339.xlsx
